### 1. Download the Data from Kaggle and Unzip


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'maedemaftouni/large-covid19-ct-slice-dataset'

In [1]:
import shutil
from google.colab import drive

drive.mount("/content/gdrive")
# Change the code below if the path to the dataset is different for you.
shutil.unpack_archive("/content/gdrive/MyDrive/archive.zip", "/tmp/")

Mounted at /content/gdrive


### 2. Splitting

In [2]:
import pandas as pd

meta_normal = pd.read_csv("/tmp/meta_data_normal.csv")
meta_covid = pd.read_csv("/tmp/meta_data_covid.csv", encoding='windows-1252')
meta_cap = pd.read_csv("/tmp/meta_data_cap.csv")

# Define the variables below using meta dataframes

normal_pt_nb = len(pd.unique(meta_normal["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in normal group
covid_pt_nb = len(pd.unique(meta_covid["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in covid group
cap_pt_nb = len(pd.unique(meta_cap["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in CAP group

normal_img_nb = len(meta_normal) ##### INSERT YOUR CODE HERE ##### # Number of images in normal group
covid_img_nb = len(meta_covid) ##### INSERT YOUR CODE HERE ##### # Number of images in covid group
cap_img_nb = len(meta_cap) ##### INSERT YOUR CODE HERE ##### # Number of images in CAP group

print("Normal")
print("Number of patient: ", normal_pt_nb)
print("Number of image: ", normal_img_nb)

print("\nCovid")
print("Number of patient: ", covid_pt_nb)
print("Number of image: ", covid_img_nb)

print("\nCAP")
print("Number of patient: ", cap_pt_nb)
print("Number of image: ", cap_img_nb)

Normal
Number of patient:  604
Number of image:  6893

Covid
Number of patient:  464
Number of image:  7593

CAP
Number of patient:  54
Number of image:  2618


In [3]:
import numpy as np

# Set seed to get the same result (I specifically chose this seed after a couple of tries so that we'll have approximately same split ratios on slice level as well)
np.random.seed(58)
val_split_size = .2
test_split_size = .5

normal_val_file_list, normal_test_file_list = [], []
covid_val_file_list, covid_test_file_list = [], []
##### START OF YOUR CODE #####
normal_pt_list = np.array(pd.unique(meta_normal["Patient ID"]))
np.random.shuffle(normal_pt_list)
normal_val_pts = normal_pt_list[:int(normal_pt_nb * val_split_size)]
normal_test_pts = normal_pt_list[int(normal_pt_nb * val_split_size): int(normal_pt_nb * (val_split_size+test_split_size))]

normal_val_file_list = []
for pt in normal_val_pts:
  for i in meta_normal["File name"][meta_normal["Patient ID"] == pt].values:
    normal_val_file_list.append(i)

normal_test_file_list = []
for pt in normal_test_pts:
  for i in meta_normal["File name"][meta_normal["Patient ID"] == pt].values:
    normal_test_file_list.append(i)

covid_pt_list = np.array(pd.unique(meta_covid["Patient ID"]))
np.random.shuffle(covid_pt_list)
covid_val_pts = covid_pt_list[:int(covid_pt_nb * val_split_size)]
covid_test_pts = covid_pt_list[int(covid_pt_nb * val_split_size): int(covid_pt_nb * (val_split_size+test_split_size))]

covid_val_file_list = []
for pt in covid_val_pts:
  for i in meta_covid["File name"][meta_covid["Patient ID"] == pt].values:
    covid_val_file_list.append(i)

covid_test_file_list = []
for pt in covid_test_pts:
  for i in meta_covid["File name"][meta_covid["Patient ID"] == pt].values:
    covid_test_file_list.append(i)

##### END OF YOUR CODE #####

print("Slice-based val size: ")
print("Normal: ", round(len(normal_val_file_list)/normal_img_nb, 2))
print("Covid: ", round(len(covid_val_file_list)/covid_img_nb, 2))

print("\nSlice-based test size: ")
print("Normal: ", round(len(normal_test_file_list)/normal_img_nb, 2))
print("Covid: ", round(len(covid_test_file_list)/covid_img_nb, 2))

Slice-based val size: 
Normal:  0.21
Covid:  0.18

Slice-based test size: 
Normal:  0.51
Covid:  0.5


In [4]:
import os

##### START OF YOUR CODE #####
# Create train and test directories and move the files accordingly
[os.makedirs("/tmp/curated_data/data/"+x+y, exist_ok=True) for x in ["train/", "val/", "test/"] for y in ["normal/", "covid/"]]

# Normal group
for pt in os.listdir("/tmp/curated_data/curated_data/1NonCOVID/"):
  # validation
  if pt in normal_val_file_list:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/val/normal/")
  # test
  elif pt in normal_test_file_list:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/test/normal/")
  # train
  else:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/train/normal/")

# Covid group
for pt in os.listdir("/tmp/curated_data/curated_data/2COVID/"):
  # validation
  if pt in covid_val_file_list:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/val/covid/")
  # test
  elif pt in covid_test_file_list:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/test/covid/")
  # train
  else:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/train/covid/")

##### END OF YOUR CODE #####

data_counts = {x+y: len(os.listdir("/tmp/curated_data/data/"+x+y)) for x in ["train/", "val/", "test/"] for y in ["normal/", "covid/"]}
for i in ["train", "val", "test"]:
  print("\nPercentage of {} set: {:.2f}" .format(i, (data_counts[i+"/normal/"]+data_counts[i+"/covid/"])/sum(data_counts.values())))
  print("Percentage of Covid + slices in {} set is: {:.2f}" .format(i, data_counts[i+"/covid/"]/(data_counts[i+"/normal/"]+data_counts[i+"/covid/"])))


Percentage of train set: 0.30
Percentage of Covid + slices in train set is: 0.55

Percentage of val set: 0.19
Percentage of Covid + slices in val set is: 0.49

Percentage of test set: 0.51
Percentage of Covid + slices in test set is: 0.52


### 3. Dataloader

In [ ]:
!pip install -q torchio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.0/173.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 12.3 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import torch
from PIL import Image
import torchio as tio
import torchvision.transforms as T


class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, image_size, data_folder, partition):
    ##### START OF YOUR CODE #####
    self.image_size = image_size
    self.partition = partition
    self.data_folder = data_folder
    self.paths = self.img_paths()
    ##### END OF YOUR CODE #####

  def __len__(self):
    ##### START OF YOUR CODE #####
    return len(self.paths)
    ##### END OF YOUR CODE #####

  def __getitem__(self, idx):

    ##### START OF YOUR CODE #####
    img = self.read_and_resize_img(self.paths[idx])
    label = int(self.paths[idx].rsplit("/", 2)[1] == "covid")
    label = np.array(label)[np.newaxis]

    if self.partition == "train":
        img = self.augmentation(img)
    return (img, label)
    ##### END OF YOUR CODE #####

  def img_paths(self):
    ##### START OF YOUR CODE #####
    normal_paths = [os.path.join(self.data_folder, self.partition, "normal", i) for i in os.listdir(os.path.join(self.data_folder, self.partition, "normal"))]
    covid_paths = [os.path.join(self.data_folder, self.partition, "covid", i) for i in os.listdir(os.path.join(self.data_folder, self.partition, "covid"))]
    paths = normal_paths + covid_paths
    np.random.shuffle(paths)
    return paths
    ##### END OF YOUR CODE #####

  def read_and_resize_img(self, path):
    ##### START OF YOUR CODE #####
    img = Image.open(path).convert('L')
    img = np.array(img)[np.newaxis]/255.

    if (self.image_size, self.image_size) != img.shape[1:]:
      resizing = T.Resize((self.image_size, self.image_size), antialias=False)
      resized_img = resizing(torch.Tensor(img))
      return resized_img
    return torch.Tensor(img)
    ##### END OF YOUR CODE #####

  def augmentation(self, data):
    ##### START OF YOUR CODE #####
    transform = tio.transforms.OneOf({
        tio.transforms.OneOf({
            tio.transforms.RandomNoise(): .25,
            tio.transforms.RandomBiasField(): .25,
            tio.transforms.RandomGhosting(): .25,
            tio.transforms.RandomSpike(): .25,
            tio.transforms.RandomAffine(degrees=10, scales=0., translation=0.): .25
        }): .8
    })
    aug_data = torch.squeeze(transform(torch.unsqueeze(data, -1)), -1)
    return aug_data
    ##### END OF YOUR CODE #####


### 4. ResNet-18

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

class ConvBlock(nn.Module):
    def __init__(self, ch_in, ch_out, s, act):
      super(ConvBlock,self).__init__()
      # Initialize layers
      if act == "relu":
          self.act_layer = nn.ReLU()
      elif act == "leaky_relu":
          self.act_layer = nn.LeakyReLU()
      elif act == "gelu":
          self.act_layer = nn.GELU()

      self.conv1 = nn.Conv2d(ch_in, ch_in, kernel_size=3, stride=s, padding=1)
      self.bn1 = nn.BatchNorm2d(ch_in)
      self.conv2 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
      self.bn2 = nn.BatchNorm2d(ch_out)

      self.conv_sc = nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=s)
      self.bn_sc = nn.BatchNorm2d(ch_out)

    def forward(self, X):
      X_shortcut = X
      X = self.bn1(self.conv1(X))
      X = self.act_layer(X)
      X = self.bn2(self.conv2(X))

      X_shortcut = self.bn_sc(self.conv_sc(X_shortcut))
      X = self.act_layer(X + X_shortcut)
      return X


class ResNet18(nn.Module):
    def __init__(self, act, drop_rate, image_size):
      super(ResNet18, self).__init__()
      if act == "relu":
          self.act_layer = nn.ReLU()
      elif act == "leaky_relu":
          self.act_layer = nn.LeakyReLU()
      elif act == "gelu":
          self.act_layer = nn.GELU()

      self.image_size = image_size

      self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3) # 64 x N/2 x N/2
      self.bn1 = nn.BatchNorm2d(64)

      self.mp = nn.MaxPool2d((3, 3), stride=2, padding=1) # 64 x N/4 x N/4
      self.conv2a = ConvBlock(64, 64, 1, act)
      self.conv2b = ConvBlock(64, 64, 1, act) # 64 x N/4 x N/4

      self.conv3a = ConvBlock(64, 128, 2, act)
      self.conv3b = ConvBlock(128, 128, 1, act) # 128 x N/8 x N/8

      self.conv4a = ConvBlock(128, 256, 2, act)
      self.conv4b = ConvBlock(256, 256, 1, act) # 256 x N/16 x N/16

      self.conv5a = ConvBlock(256, 512, 2, act)
      self.conv5b = ConvBlock(512, 512, 1, act) # 512 x N/32 x N/32

      kernel_size = self.image_size /(2**5)
      self.avgpool = nn.AvgPool2d((int(kernel_size), int(kernel_size)))
      self.flat = nn.Flatten()
      self.dropout = nn.Dropout(drop_rate)
      self.fc = nn.Linear(512, 1)

    def forward(self, X):
      batch_size = X.size(0)
      image_size = X.size(2)

      X = self.conv1(X)
      X = self.bn1(X)
      X = self.act_layer(X)

      X = self.mp(X)
      X = self.conv2a(X)
      X = self.conv2b(X)

      X = self.conv3a(X)
      X = self.conv3b(X)

      X = self.conv4a(X)
      X = self.conv4b(X)

      X = self.conv5a(X)
      X = self.conv5b(X)

      X = self.avgpool(X)
      X = self.flat(X)
      X = self.fc(self.dropout(X))
      X = torch.sigmoid(X)
      return X

# Assignment 3
<p>In this assignment you will

* write helper functions
* train the model
* hyperparameter search using W&B

Read the comments carefully and insert your code where you see: <br><br><b>##### START OF YOUR CODE #####</b><br><br><b>##### END OF YOUR CODE #####</b><br><br>or for the inline codes you will see<br><br><b>##### INSERT YOUR CODE HERE #####</b>

#### I. AverageMeter
First we will write a helper function. AverageMeter is to calculate the mean of the running loss and accuracy.

*   It will have 2 functions which are reset and update.
*   reset will be called on initialization and set the attributes to 0.
*   update takes 2 arguments for the value and the size. It will add the value to the sum and the size to the count. Attribute "avg" (use this name) will also be updated as sum/count.

In [ ]:
class AverageMeter:
    """Computes and stores the average and current value"""

    def __init__(self):
      ##### START OF YOUR CODE #####
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
      ##### END OF YOUR CODE #####

In [ ]:
avg_meter = AverageMeter()
avg_meter.update(100, 5)
avg_meter.update(50, 5)
print(avg_meter.avg, avg_meter.count)

75.0 10


#### II. Train Loop
Now we will write the training and validation loops. Detailed instructions are given within the code.

In [ ]:
from tqdm import tqdm

device = torch.device('cuda')

def training(train_loader, model, criterion, optimizer):
  # Let's start by initializing our AverageMeters.
  avg_meters = {'loss': AverageMeter(),
                'acc': AverageMeter()}

  # We will go through the train_loader.
  # Zero the gradients.
  # Make prediction.
  # Calculate the loss and the accuracy using prediction and labels.
  # Update the average meters.
  # Compute gradients and adjust learning weights.

  ##### START OF YOUR CODE #####
  model.train()
  for image, label in tqdm(train_loader):
    image = image.to(device)
    label = label.to(device)
    optimizer.zero_grad()
    output = model(image)
    loss = criterion(output, label.float())
    loss.backward()

    optimizer.step()

    output = output > .5
    acc = (label == output).float().sum().item() / label.size(0)
    avg_meters['loss'].update(loss.item(), image.size(0))
    avg_meters['acc'].update(acc, image.size(0))

  ##### END OF YOUR CODE #####

  return dict([('loss', avg_meters['loss'].avg),
                ('acc', avg_meters['acc'].avg)])

def validation(val_loader, model, criterion):
  avg_meters = {'loss': AverageMeter(),
                'acc': AverageMeter()}

  # Validation is almost the same but don't forget to turn the eval mode of the model and with torch no_grad.
  # You don't need to compute gradients or adjust learning weights for evaluation.

  ##### START OF YOUR CODE #####
  model.to(device)
  model.eval()
  with torch.no_grad():
    for image, label in tqdm(val_loader):
      image =image.to(device)
      label = label.to(device)
      output = model(image)
      loss = criterion(output, label.float())

      output = output > .5
      acc = (label == output).float().sum().item() / label.size(0)
      avg_meters['loss'].update(loss.item(), image.size(0))
      avg_meters['acc'].update(acc, image.size(0))

  ##### END OF YOUR CODE #####

  return dict([('loss', avg_meters['loss'].avg),
              ('acc', avg_meters['acc'].avg)])

We will use Weights & Biases for hyperparameter search. This will only be an introduction and we highly recommend you to read the <a href="https://docs.wandb.ai/?_gl=1*1xon9b*_ga*NDg5OTYzNTM3LjE2NzUwNjYzNjk.*_ga_JH1SJHJQXJ*MTY3Njc0MDEyNi4xMi4xLjE2NzY3NDAxMjguNTguMC4w">documentation</a> for more information.

In [ ]:
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [ ]:
from torch import optim
import wandb
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

def main():
  # Set the initial configuration
  initial_config = {
      "data_dir": "/tmp/curated_data/data/",
      "image_size": 128,
      "train_batch_size": 64,
      "val_batch_size": 32,
      "test_batch_size": 1,
      "activation": "relu",
      "drop_rate": .2,
      "optimizer": "Adam",
      "learning_rate": 1e-3,
      "l2_reg": 1e-4, # Weight decay
      "nb_epoch": 50,
      "early_stopping": 15, # trigger value for early stopping

  }

  # Using this configuration dictionary:
  # initialize wandb
  # Create a run directory in your drive ("/content/drive/MyDrive/CMPE_runs/" + the current run name that you'll get from wandb)
  # Create the model
  # Create dataloader dictionary with "train", "val", "test" keys
  # Define binary cross entropy loss
  # Define optimizer with weight decay
  # Set lr scheduler to ReduceLROnPlateau:
    # It will decrease the lr by .1 if the val_loss did not decrease > .01. The minimum lr value can be 1e-9.
  # Print train and val results and log them to wandb at the end of each epoch
  # Save best model weights to your run directory when the val accuracy is at least .01 better than the best val accuracy.
  # Set early stopping with the trigger in config["early_stopping"], monitoring val accuracy. config["early_stopping"] = -1 means no early stopping.
  # Print when a new model is saved or early stopping trigger is reached.
  # After the final epoch (or early stopping), load the best model weights and log the test results to wandb

  ##### START OF YOUR CODE #####
  wandb.init(config=initial_config)
  config = wandb.config

  run_dir = os.path.join("/content/drive/MyDrive/CMPE_runs/", wandb.run.name)
  os.makedirs(run_dir, exist_ok=True)

  model = ResNet18(config["activation"], config["drop_rate"], config["image_size"])
  model.to(device)

  data_dict = {x: CustomDataset(config["image_size"], config["data_dir"], x) for x in ["train", "val", "test"]}
  dataloader_dict = {x: DataLoader(data_dict[x], config[x+"_batch_size"]) for x in ["train", "val", "test"]}
  criterion =  nn.BCELoss()


  params =  filter(lambda p: p.requires_grad, model.parameters())
  if config['optimizer'] == 'Adam':
      optimizer =  optim.Adam(params, lr=config['learning_rate'], weight_decay=config["l2_reg"])
  elif config['optimizer'] == 'SGD':
      optimizer =  optim.SGD(params, lr=config['learning_rate'], weight_decay=config["l2_reg"],)
  elif config['optimizer'] == 'RMSprop':
      optimizer =  optim.RMSprop(params, lr=config['learning_rate'], weight_decay=config["l2_reg"])

  scheduler =  optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=.1, patience=10, threshold_mode="abs", threshold=0.01, verbose=1, min_lr=1e-9)

  best_acc, trigger = 0, 0
  for e in range(config["nb_epoch"]):
    print('Epoch [%d/%d]' % (e+1, config['nb_epoch']))

    train_log = training(dataloader_dict["train"], model, criterion, optimizer)
    val_log = validation(dataloader_dict["val"], model, criterion)


    scheduler.step(val_log['loss'])

    print('loss %.4f - acc %.4f - val_loss %.4f - val_acc %.4f' % (train_log['loss'], train_log['acc'], val_log['loss'], val_log['acc']))
    wandb.log({"train_"+x: train_log[x] for x in train_log.keys()})
    wandb.log({"val_"+x: val_log[x] for x in val_log.keys()})

    trigger += 1
    if val_log['acc'] - best_acc > .01:
        torch.save(model.state_dict(), os.path.join(run_dir, "best_model.pth"))
        best_acc = val_log['acc']
        print("=> saved best model")
        trigger = 0

    # early stopping
    if config['early_stopping'] >= 0 and trigger >= config['early_stopping']:
        print("=> early stopping")
        break
  model.load_state_dict(torch.load(os.path.join(run_dir, "best_model.pth")))
  test_log = validation(dataloader_dict["test"], model, criterion)
  wandb.log({"test_"+x: x for x in test_log.keys()})
  ##### END OF YOUR CODE #####

In [ ]:
# Define the parameters that we will fine tune with, which are:
  # Activation function
  # Optimizer
  # Drop rate: should be chosen randomly from a uniform distribution between [0., 0.9]
  # Weight decay: should be chosen randomly from a uniform distribution between [0., 0.1]
  # Learning rate: should be chosen randomly from a uniform distribution between [0.0001, 0.1]

parameter_dict = {
    'activation': {
        'values': ['leaky_relu']
    },
    'optimizer': {
        'values': ['SGD']
    },
    'drop_rate': {
        'distribution': 'uniform',
        'min': 0.5,  # Narrowed down the range
        'max': 0.9
    },
    'l2_reg': {
        'distribution': 'uniform',
        'min': 0.02,  # Narrowed down the range
        'max': 0.05
    },
    'learning_rate': {
        'distribution': 'uniform',
        'min': 0.05,  # Narrowed down the range
        'max': 0.1
    }
}

# Define a sweep configuration tells wandb that it will randomly choose from parameter dict and the purpose is to maximize val_accuracy
sweep_config = {
    ##### START OF YOUR CODE #####
    'method': 'random',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    }
    ##### END OF YOUR CODE #####
}
sweep_config['parameters'] = parameter_dict

# Start the sweep
sweep_id = wandb.sweep(sweep_config, project='CMPE49T') ##### INSERT YOUR CODE HERE #####

 33%|███▎      | 23/69 [00:11<00:21,  2.09it/s]

Create sweep with ID: m0v9l18v
Sweep URL: https://wandb.ai/bounteam/CMPE49T/sweeps/m0v9l18v


 35%|███▍      | 24/69 [00:12<00:20,  2.16it/s]

In [ ]:
wandb.agent(sweep_id, function=main)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
 80%|███████▉  | 55/69 [00:28<00:06,  2.11it/s]wandb: Agent Starting Run: nxssyfn6 with config:
wandb: 	activation: leaky_relu
wandb: 	drop_rate: 0.7959947005859069
wandb: 	l2_reg: 0.023380302123292063
wandb: 	learning_rate: 0.05192433093654631
wandb: 	optimizer: SGD
 90%|████████▉ | 62/69 [00:32<00:03,  1.92it/s]

Epoch [1/50]



100%|██████████| 88/88 [00:24<00:00,  3.59it/s]


loss 0.4204 - acc 0.8229 - val_loss 0.6463 - val_acc 0.5589
Epoch [21/50]


100%|██████████| 69/69 [00:57<00:00,  1.20it/s]

100%|██████████| 88/88 [00:24<00:00,  3.57it/s]


...

100%|██████████| 88/88 [00:14<00:00,  6.03it/s]


loss 0.0919 - acc 0.9760 - val_loss 0.5767 - val_acc 0.7804
Epoch [16/50]


100%|██████████| 88/88 [00:14<00:00,  6.02it/s]


loss 0.0647 - acc 0.9890 - val_loss 0.5888 - val_acc 0.7753
Epoch [17/50]


 30%|███       | 21/69 [00:10<00:23,  2.01it/s]